# 25 - Multiple Linear Regression analysis

ANOVA - Analysis of variance

## Import libraries

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import geopandas as gpd

import seaborn as sns
import matplotlib.pyplot as plt

import os
import sys
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/edgaregurrola/Repos/observatorio-ciudades/accesibilidad-urbana/aup/data.py:24: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


## Config notebook

## Config notebook

In [2]:
##### WARNING ##### WARNING ##### WARNING ##### WARNING ##### WARNING #####

# save output to database?
save = False
# How to save?
if_exists='append'
# save to local?
save_local = False
# use local data?
local_data = False
##### WARNING ##### WARNING ##### WARNING ##### WARNING ##### WARNING #####

In [3]:
# 'alex' or 'edgar'
all_local_dirs = 'edgar'
# Output hexs resolution
res = 10
# Persona speed
walk_speed = 4.5
speed_name = str(walk_speed).replace('.','_')

In [4]:
if all_local_dirs == 'alex':
    grl_dir = "../../../data/external/temporal_todocker/santiago/proximidad/"
elif all_local_dirs == 'edgar':
    grl_dir = "../../../data/processed/santiago/"
else:
    print("WARNING: Fix all_local_dirs variable")

## __Load data__

### __Load data__ - Pois proximity (regular and unique ID pois)

In [5]:
if local_data:
    hex_gdf = gpd.read_file(grl_dir + f'santiago_hexanalysis_{speed_name}_kmh_res{res}.gpkg')
else:
    query = f'SELECT * FROM projects_research.santiago_hexanalysis_{speed_name}_kmh WHERE res = {res}'
    hex_gdf = aup.gdf_from_query(query)

# Show
print(hex_gdf.shape)
hex_gdf.head(1)

(58695, 31)


,wellbeing_supplying,wellbeing_caring,wellbeing_living,wellbeing_enjoying,wellbeing_learning,wellbeing_working,wellbeing,sociability_supplying,sociability_caring,sociability_living,...,hex_id,geometry,supplying,caring,living,enjoying,learning,working,hqsl,res
0,5.833333,5.833333,5.0,5.638199,7.5,4.166667,33.971533,4.583333,1.875,0.543189,...,8ab2c5540647fff,"POLYGON ((-70.67723 -33.43729, -70.67786 -33.4...",5.87963,5.25712,3.59773,4.894993,5.833333,4.943534,30.406339,10


### __Load data__ - Comunas

In [6]:
# From notebook 12-Areal-data-to-hex.ipynb
if all_local_dirs == 'alex':
    comuna_gdf = gpd.read_file(grl_dir + 'santiago_comunas.geojson')
elif all_local_dirs == 'edgar':
    comuna_gdf = gpd.read_file(grl_dir + 'santiago_comunas.geojson')

# Show
comuna_gdf = comuna_gdf.explode()
comuna_gdf['area_ha'] = comuna_gdf.to_crs("EPSG:32719").area/10000
comuna_gdf = comuna_gdf[['Comuna','area_ha','geometry']]
comuna_gdf = comuna_gdf.rename(columns={'Comuna':'Nombre'})
comuna_gdf = comuna_gdf.reset_index(drop=True)
print(comuna_gdf.shape)
comuna_gdf.head(1)

(36, 3)


,Nombre,area_ha,geometry
0,San Joaquín,987.687671,"POLYGON ((-70.62342 -33.47338, -70.62292 -33.4..."


### __Load data__ - Buffer Alameda

In [7]:
# From notebook 12-Areal-data-to-hex.ipynb
if all_local_dirs == 'alex':
    alameda_gdf = gpd.read_file(grl_dir + 'unidades_vecinales_buffer800m_alameda_v2.gpkg')
elif all_local_dirs == 'edgar':
    alameda_gdf = gpd.read_file(grl_dir + 'unidades_vecinales_buffer800m_alameda_v2.gpkg')

# Show
alameda_gdf = alameda_gdf.dissolve()
alameda_gdf['area_ha'] = alameda_gdf.to_crs("EPSG:32719").area/10000
alameda_gdf['Nombre'] = 'Nueva Alameda'
alameda_gdf = alameda_gdf[['Nombre','area_ha','geometry']]
print(alameda_gdf.shape)
alameda_gdf.head(1)

(1, 3)


,Nombre,area_ha,geometry
0,Nueva Alameda,2266.840098,"POLYGON ((-70.71634 -33.46959, -70.71651 -33.4..."


## Multiple Linear Regression

### Data by area of analysis

In [8]:
nombre_comuna = ['Las Condes','Pedro Aguirre Cerda',
                'Quinta Normal','San Miguel',
                'Maipú','Conchalí','El Bosque',
                'Providencia']
comuna_analysis = comuna_gdf.loc[comuna_gdf.Nombre.isin(nombre_comuna)].copy()
comparison_gdf = pd.concat([comuna_analysis,alameda_gdf])
comparison_gdf

,Nombre,area_ha,geometry
1,San Miguel,963.502832,"POLYGON ((-70.63659 -33.51873, -70.63740 -33.5..."
6,Providencia,1439.652277,"POLYGON ((-70.58484 -33.43126, -70.58445 -33.4..."
17,Conchalí,1098.672380,"POLYGON ((-70.66314 -33.39923, -70.66400 -33.3..."
18,El Bosque,1426.967489,"POLYGON ((-70.66324 -33.58516, -70.66340 -33.5..."
23,Las Condes,9839.607402,"POLYGON ((-70.53235 -33.38217, -70.53235 -33.3..."
28,Maipú,13803.803111,"POLYGON ((-70.76505 -33.45926, -70.76505 -33.4..."
30,Pedro Aguirre Cerda,850.130544,"POLYGON ((-70.67123 -33.51069, -70.67179 -33.5..."
33,Quinta Normal,1186.605299,"POLYGON ((-70.70527 -33.40762, -70.70463 -33.4..."
0,Nueva Alameda,2266.840098,"POLYGON ((-70.71634 -33.46959, -70.71651 -33.4..."


### Spatial intersection

In [9]:
hex_analysis = hex_gdf.merge(hex_gdf.sjoin(comparison_gdf)[['hex_id','Nombre']], on='hex_id',
                             how='right')
print(hex_analysis.shape)
hex_analysis.head(4)

(15694, 32)


,wellbeing_supplying,wellbeing_caring,wellbeing_living,wellbeing_enjoying,wellbeing_learning,wellbeing_working,wellbeing,sociability_supplying,sociability_caring,sociability_living,...,geometry,supplying,caring,living,enjoying,learning,working,hqsl,res,Nombre
0,5.833333,5.833333,5.000000,5.638199,7.500000,4.166667,33.971533,4.583333,1.875000,0.543189,...,"POLYGON ((-70.67723 -33.43729, -70.67786 -33.4...",5.879630,5.257120,3.597730,4.894993,5.833333,4.943534,30.406339,10,Nueva Alameda
1,2.845833,2.532738,2.836364,4.570114,1.393876,3.750000,17.928926,3.202652,3.052083,0.134702,...,"POLYGON ((-70.54047 -33.41159, -70.54110 -33.4...",3.512286,3.050925,2.731769,5.760304,1.046366,2.500000,18.601651,10,Las Condes
2,0.729167,1.497396,0.000000,2.500000,2.173611,3.333333,10.233507,1.250000,4.479167,0.116598,...,"POLYGON ((-70.69534 -33.46458, -70.69597 -33.4...",0.914352,2.983119,0.969422,2.403274,1.890432,1.732263,10.892862,10,Nueva Alameda
3,1.250000,3.080357,4.250000,0.406806,3.952381,4.166667,17.106211,2.678571,5.000000,0.543317,...,"POLYGON ((-70.67531 -33.57591, -70.67594 -33.5...",2.248677,3.723022,3.653328,2.227213,3.141534,2.916667,17.910441,10,El Bosque


In [14]:
hex_analysis.columns

Index(['wellbeing_supplying', 'wellbeing_caring', 'wellbeing_living',
       'wellbeing_enjoying', 'wellbeing_learning', 'wellbeing_working',
       'wellbeing', 'sociability_supplying', 'sociability_caring',
       'sociability_living', 'sociability_enjoying', 'sociability_learning',
       'sociability_working', 'sociability', 'environmental_impact_supplying',
       'environmental_impact_caring', 'environmental_impact_living',
       'environmental_impact_enjoying', 'environmental_impact_learning',
       'environmental_impact_working', 'environmental_impact', 'hex_id',
       'geometry', 'supplying', 'caring', 'living', 'enjoying', 'learning',
       'working', 'hqsl', 'res', 'Nombre'],
      dtype='object')

### Distribution analysis

In [10]:
variable_analysis = 'environmental_impact_living'

Count data

In [11]:
hex_analysis[['Nombre',variable_analysis]].groupby('Nombre').count()

,environmental_impact_living
Nombre,
Conchalí,916
El Bosque,1132
Las Condes,3661
Maipú,4329
Nueva Alameda,1931
Pedro Aguirre Cerda,723
Providencia,1194
Quinta Normal,1003
San Miguel,805


In [13]:
hex_mlr = hex_analysis.groupby('Nombre').apply(lambda x: x.sample(n=500), include_groups=False).reset_index().drop(columns='level_1')
hex_mlr[['Nombre',variable_analysis]].groupby('Nombre').count()

,environmental_impact_living
Nombre,
Conchalí,500
El Bosque,500
Las Condes,500
Maipú,500
Nueva Alameda,500
Pedro Aguirre Cerda,500
Providencia,500
Quinta Normal,500
San Miguel,500


### Model creation

In [21]:
area_analysis = 'Nueva Alameda'
Y = hex_mlr.loc[hex_mlr.Nombre==area_analysis, hex_mlr.columns=='hqsl']
X = hex_mlr.loc[hex_mlr.Nombre==area_analysis, 
hex_mlr.drop(columns=['hqsl','hex_id','Nombre','res','geometry']).columns]

In [23]:
X

,wellbeing_supplying,wellbeing_caring,wellbeing_living,wellbeing_enjoying,wellbeing_learning,wellbeing_working,wellbeing,sociability_supplying,sociability_caring,sociability_living,...,environmental_impact_enjoying,environmental_impact_learning,environmental_impact_working,environmental_impact,supplying,caring,living,enjoying,learning,working
2000,6.583333,3.906250,4.687500,7.309573,8.395833,8.020833,38.903323,7.812500,5.625000,0.140545,...,3.750000,0.0,7.187500,27.135086,7.372685,3.831542,3.146385,6.545411,4.803241,6.533307
2001,5.500000,4.270833,4.750000,5.715390,8.777778,8.194444,37.208446,7.916667,5.000000,0.692136,...,1.250000,0.0,8.333333,27.374514,6.805556,4.284919,3.332379,5.752040,5.015432,6.993935
2002,7.333333,7.500000,8.125000,7.918244,6.750000,5.277778,42.904355,7.083333,5.000000,0.472488,...,10.000000,0.0,10.000000,43.378210,7.546296,7.002023,5.378726,8.620580,5.000000,6.932706
2003,1.500000,0.625000,2.250000,3.631548,5.166667,4.166667,17.339881,4.375000,5.000000,1.969013,...,5.000000,0.0,1.250000,25.552239,2.708333,2.329271,2.914850,4.382738,4.675926,2.609034
2004,1.666667,2.109375,1.250000,8.444417,4.416667,2.738095,20.625221,2.500000,2.708333,1.624970,...,7.916667,0.0,9.791667,35.970006,2.870370,2.703855,1.589921,7.164917,2.453704,4.708560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,4.000000,1.718750,0.500000,3.813155,6.500000,7.708333,24.240238,7.083333,3.333333,2.451938,...,3.750000,0.0,2.812500,20.397725,4.930556,2.405593,1.815646,4.172202,3.888889,5.569069
2496,5.250000,8.125000,8.250000,5.629588,4.166667,4.166667,35.587921,3.750000,3.541667,0.913683,...,8.333333,0.0,10.000000,40.350143,5.277778,7.061829,5.519839,6.408544,2.453704,6.380345
2497,6.583333,3.281250,6.500000,7.076783,8.354167,8.020833,39.816367,7.708333,6.250000,0.600296,...,2.187500,0.0,6.406250,26.289816,7.106481,4.098650,4.318154,6.200222,4.780093,7.419587
2498,7.797619,7.656250,10.000000,7.806519,6.535714,5.158730,44.954833,6.666667,7.916667,1.934631,...,10.000000,0.0,8.035714,41.756597,7.665344,7.619651,6.927615,8.501474,4.880952,6.924603


In [25]:
logreg = LinearRegression()
logreg.fit(X, Y)
Y_pred = logreg.predict(X)
lin_mse = mean_squared_error(Y,Y_pred)
lin_rmse = np.sqrt(lin_mse)

In [29]:
print(logreg.coef_)

[[ 0.07614562  0.15988202  0.07614562  0.02817494  0.07494188  0.02685
   0.44214007  0.08357658 -0.07688931  0.08357658  0.12534143  0.08285434
  -0.0710648   0.22739483 -0.02768596  0.01916242 -0.02768596 -0.01376435
   0.          0.18131695  0.1313431   0.06708576  0.09696686  0.06708576
   0.05936997  0.06925249  0.06201985]]


In [32]:
from statsmodels.api import OLS
OLS(Y,X[['wellbeing','sociability','environmental_impact']]).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   hqsl   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          4.317e+05
Date:                Thu, 11 Jul 2024   Prob (F-statistic):                        0.00
Time:                        06:51:34   Log-Likelihood:                         -425.29
No. Observations:                 500   AIC:                                      856.6
Df Residuals:                     497   BIC:                                      869.2
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
wellbeing                0.6357      0.004    150.708      0.000       0.627       0.644
sociability              0.2235      0.005     42.169      0.000       0.213       0.234
environmental_impact     0.1350      0.003     41.369      0.000       0.129       0.141
==============================================================================
Omnibus:                        1.403   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.496   Jarque-Bera (JB):                1.215
Skew:                           0.060   Prob(JB):                        0.545
Kurtosis:                       3.210   Cond. No.                         12.1
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""